In [1]:
#import necessary package 
from bs4 import BeautifulSoup
import requests

In [2]:
#picking a random header to avoid the block.
import random
headerlist = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
     "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991",
           "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36 OPR/42.0.2393.94",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36 OPR/47.0.2631.39",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
           "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:56.0) Gecko/20100101 Firefox/56.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko"]
user_agent = random.choice(headerlist)
fake_headers = {'User-Agent': user_agent}
#Remark:I find out that different fake agent will create different searrch result.

In [3]:
#downloading the original html
booking_url="https://www.booking.com/searchresults.html"
my_params={"ss":"Taipei","checkin":"2023-10-28","checkout":"2023-10-29"}
r=requests.get(booking_url , params=my_params,headers=fake_headers)
if r.status_code == requests.codes.ok:
    print("Ok")
    print(r.url)
    soup=BeautifulSoup(r.text,"html.parser")


Ok
https://www.booking.com/searchresults.html?ss=Taipei&checkin=2023-10-28&checkout=2023-10-29


In [4]:
import pandas as pd
hotel_info=pd.DataFrame(columns=["name","location","price","rating","distance","comments"])
hotel_info

,name,location,price,rating,distance,comments


In [5]:
#finding hotel name
hotel_name=soup.find_all(class_="f6431b446c a15b38c233")
hotel_name_Series=pd.Series(len(hotel_name))
for i in range(len(hotel_name)):
    hotel_name_Series[i]=hotel_name[i].string
hotel_info["name"]=hotel_name_Series

In [6]:
#finding hotel location
hotel_location=soup.find_all(attrs={"data-testid": "address"},class_="aee5343fdb def9bc142a")
hotel_location_Series=pd.Series(len(hotel_location))
for i in range(len(hotel_location)):
    hotel_location_Series[i]=hotel_location[i].string
hotel_info["location"]=hotel_location_Series

In [7]:
#finding hotel price
hotel_price=soup.find_all(attrs={"data-testid": "price-and-discounted-price"},class_="f6431b446c fbfd7c1165 e84eb96b1f")
hotel_price_Series=pd.Series(len(hotel_price))
for i in range(len(hotel_price)):
    hotel_price_Series[i]=hotel_price[i].string
hotel_info["price"]=hotel_price_Series

In [12]:
hotel_info

,name,location,price,rating,distance,comments
0,Happy Taipei Hostel - Share House,"Shilin District , Taipei",0.0,8.5,5 km from centre,Very good
1,苓旅中山館-Lininn ZhongShan,"Zhongshan District, Taipei",NaN,8.6,2.4 km from centre,Fabulous
2,Sunrise Hotel,"Xinyi District, Taipei",NaN,7.8,5 km from centre,Good
3,NIA NIA Ximending 西門町,"Wanhua District , Taipei",NaN,NaN,1.2 km from centre,NaN
4,苓旅-站前 Lin Inn TMS,"Zhongzheng District, Taipei",NaN,8.1,350 m from centre,Very good
5,W S Stay,"Datong District , Taipei",NaN,7.9,0.8 km from centre,Good
6,苓旅萬年-LIN INN Wan Nian,"Wanhua District , Taipei",NaN,8.0,1.2 km from centre,Very good
7,The Blue by Just Inn 正旅館 藍,"Zhongzheng District, Taipei",NaN,NaN,300 m from centre,NaN
8,Welove meworld Inn,"Wanhua District , Taipei",NaN,7.7,1.4 km from centre,Good
9,熙門小聚 YS Inn,"Zhongzheng District, Taipei",NaN,7.7,0.8 km from centre,Good


In [8]:
#finding hotel score
#remark: some results have no score since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external score.
#treatment: it is not pretty fair to compare the external score with booking.com internal score, so there I treat all the newcomers without internal score as nan.  
from math import nan #in order to assign nana value to the hotel with no internal score
hotel_score=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
hotel_score_Series=pd.Series(len(hotel_score))
print(len(hotel_score))
for i in range(len(hotel_score)):
    sub_soup = BeautifulSoup(str(hotel_score[i]),'html.parser')
    try:
        hotel_score_Series[i]=float(sub_soup.find(class_="a3b8729ab1 d86cee9b25").text)
    except:
        hotel_score_Series[i]= nan
#common class for both types
#with score
#aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
#without score
#aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1
hotel_info["rating"]=hotel_score_Series

25


In [9]:
#finding hotel distance
hotel_distance=soup.find_all(attrs={"data-testid": "distance"})
hotel_distance_Series=pd.Series(len(hotel_distance))
for i in range(len(hotel_distance)):
    hotel_distance_Series[i]=hotel_distance[i].string
hotel_info["distance"]=hotel_distance_Series

In [10]:
#finding hotel comments
#remark: some results have no comments since the hotel is new to booking. However, for some of the newcomers, booking.com provide a external comments.
#treatment: it is not pretty fair to compare the external comments with booking.com internal comments, so there I treat all the newcomers without internal comments as nan.  
hotel_comments=soup.find_all(class_="aca0ade214 ebac6e22e9 cd2e7d62b0 a0ff1335a1")
hotel_comments_Series=pd.Series(len(hotel_comments))
print(len(hotel_comments))
for i in range(len(hotel_comments)):
    sub_soup = BeautifulSoup(str(hotel_comments[i]),'html.parser')
    try:
        hotel_comments_Series[i]=sub_soup.find(class_="a3b8729ab1 e6208ee469 cb2cbb3ccb").text
    except:
        hotel_comments_Series[i]= nan
hotel_info["comments"]=hotel_comments_Series

25


In [11]:
hotel_info

,name,location,price,rating,distance,comments
0,Happy Taipei Hostel - Share House,"Shilin District , Taipei",0.0,8.5,5 km from centre,Very good
1,苓旅中山館-Lininn ZhongShan,"Zhongshan District, Taipei",NaN,8.6,2.4 km from centre,Fabulous
2,Sunrise Hotel,"Xinyi District, Taipei",NaN,7.8,5 km from centre,Good
3,NIA NIA Ximending 西門町,"Wanhua District , Taipei",NaN,NaN,1.2 km from centre,NaN
4,苓旅-站前 Lin Inn TMS,"Zhongzheng District, Taipei",NaN,8.1,350 m from centre,Very good
5,W S Stay,"Datong District , Taipei",NaN,7.9,0.8 km from centre,Good
6,苓旅萬年-LIN INN Wan Nian,"Wanhua District , Taipei",NaN,8.0,1.2 km from centre,Very good
7,The Blue by Just Inn 正旅館 藍,"Zhongzheng District, Taipei",NaN,NaN,300 m from centre,NaN
8,Welove meworld Inn,"Wanhua District , Taipei",NaN,7.7,1.4 km from centre,Good
9,熙門小聚 YS Inn,"Zhongzheng District, Taipei",NaN,7.7,0.8 km from centre,Good
